### 01_preprocess로 생성된 DataFrame으로 visualization을 수행합니다. 01 과정을 먼저 수행 후 이 과정을 수행해야합니다.

In [1]:
import glob
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from ipywidgets import widgets
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
file_name = glob.glob('./dataframe/soc-gemmini*')
file_name = sorted(file_name)
file_name

['./dataframe/soc-gemmini-multisim.2MB-gemmini.basic',
 './dataframe/soc-gemmini-multisim.2MB-gemmini.double',
 './dataframe/soc-gemmini-multisim.2MB-gemmini.loop',
 './dataframe/soc-gemmini-multisim.gemmini.basic',
 './dataframe/soc-gemmini-multisim.gemmini.double',
 './dataframe/soc-gemmini-multisim.gemmini.loop']

In [3]:
if not os.path.isfile("all_result"):
    df={}
    df['2mb']={}
    df['base']={}
    for i in range(len(file_name)):
        print(file_name[i])
        with open(file_name[i], 'rb') as f:
            dataframe = pickle.load(f)
            
        if(file_name[i].split('.')[-2][:3]=="2MB"):
            df['2mb'][file_name[i].split('.')[-1]] = dataframe
        else:
            df['base'][file_name[i].split('.')[-1]] = dataframe

    with open('all_result', 'wb') as f:
        pickle.dump(df, f)
else:
    with open('all_result', 'rb') as f:
        df = pickle.load(f)


In [4]:
if not os.path.isfile("all_logs"):
	dir_path = "/home/centos/firesim/deploy/results-workload/allocate-fpga-print/version3/"
	log_paths = glob.glob(dir_path+'/*/result.log')
	log_paths = sorted(log_paths)

	logs={}
	logs['2mb']={}
	logs['base']={}

	for log_path in log_paths:
		with open(log_path, 'r') as f:
			lines = f.readlines()
		lines = lines[5:-14]

		layer_type_list=[]
		layer_num_list=[]
		start_cycle_list=[]
		end_cycle_list=[]

		for line in lines:
			layer_type, layer_num, _, start_cycle, end_cycle, _ = line.split(' ')
			layer_type_list.append(layer_type)
			layer_num_list.append(int(layer_num))
			start_cycle_list.append(int(start_cycle.split('\t')[-2]))
			end_cycle_list.append(int(end_cycle.split('\t')[-2]))
		log_df = pd.DataFrame({'layer_num':layer_num_list, "layer_type":layer_type_list,"start_cycle":start_cycle_list, "end_cycle":end_cycle_list})
		
		log_name = log_path.split('/')[-2]
		if(log_name.split('.')[-2][:3]=="2MB"):
			logs['2mb'][log_name.split('.')[-1]] = log_df
		else:
			logs['base'][log_name.split('.')[-1]] = log_df
			
	with open('all_logs', 'wb') as f:
		pickle.dump(logs, f)

else:
	with open('all_logs', 'rb') as f:
		logs = pickle.load(f)

In [ ]:
#select config(hw, sw, layer)
hardware = 'base'
software = 'loop'
layer_num = 3

log_df = logs[hardware][software]
result_df = df[hardware][software]


#cycle config : 너무 많은 사이클을 나타내려고 하면 visualization이 안될 수 있습니다.
start_cycle = log_df[log_df.layer_num==layer_num].start_cycle.values[0]
end_cycle = start_cycle + 100000
# end_cycle = log_df[log_df.layer_num==layer_num].end_cycle.values[0]



layer_df = result_df[np.logical_and(result_df.cycle>=start_cycle, result_df.cycle<=end_cycle)]

fig = px.line(layer_df, x='row_addr',y='bank', color='control',
		category_orders={'bank':[3,2,1,0]}, hover_data=['cycle'], symbol='control'
)
fig.update_xaxes(range=[0,4096])
fig.update_yaxes(range=[0,3])
fig.update_yaxes(type='category')
fig.update_layout(width=1000, height=400)

# plot show or save
fig.show()
# fig.write_html("{}_{}_{}.html".format(hardware,software,layer_num))  #save html
# fig.write_image("fig1.png") # save image

#execute control visualize
excute_bank = layer_df[layer_df.control=="E"].bank
plt.plot(excute_bank)